In [14]:
!pip install apyori

In [15]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
# Colab Notebooks 안에 my_env 폴더에 패키지 저장
os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
sys.path.insert(0, my_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileExistsError: ignored

In [16]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm 
fm._rebuild() 
%matplotlib inline

'''# 그래프에서 한글표현을 위해 폰트를 설치합니다.
%config InlineBackend.figure_format = 'retina'

!apt -qq -y install fonts-nanum > /dev/null
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)'''

import feedparser
from bs4 import BeautifulSoup as bs
import urllib
import urllib.request as req
import requests

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

from konlpy.tag import Kkma, Okt, Komoran
okt = Okt()
from konlpy.utils import pprint

from datetime import datetime

import re

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
import mlxtend

In [90]:
!pip install pyfpgrowth

     |████████████████████████████████| 1.6MB 2.8MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5479 sha256=d0327d834e6593e6755e70ce8f43efc0d28ca6c3e2cd5e8adacd181fca8738e0
  Stored in directory: /root/.cache/pip/wheels/3b/3f/0d/a04bb8b17887c1eca7d0f1a48d4aa0c09c96eb221ff7fa56c1
Successfully built pyfpgrowth


In [91]:
import pyfpgrowth

In [18]:
href_list = []   # 기사 주소가 들어갈 리스트
TitDesc_list = []   # 제목 + 요약 내용 리스트
title_list = []   # 제목 리스트

In [19]:
def Crawling(keyword, date):
    keyword = '+'.join(keyword.split(' '))

    last = False
    page_num = 1

    ds = date
    de = ds
    while last == False:
        url = "https://search.naver.com/search.naver?&where=news&query={0}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={1}&de={2}&docid=&nso=so:r,p:,a:all&mynews=1&cluster_rank=238&start={3}&refresh_start=0".format(keyword,ds,de,str(page_num))
        raw = requests.get(url)
        html = raw.content
        soup = bs(html, 'html.parser')

        ul = soup.find('ul', {'class':'type01'})
        li_list = ul.findAll('li')

        for li in li_list:
            try:
                href_list.append(li.dl.dt.a['href'])

                d_list = li.findAll('dd')
                #자식 노드 dd가 두 개인데 두 번째 노드에 description이 들어가있으므로 전부 불러온 후 인덱싱할 것임

                title = li.dl.dt.a['title']
                description = d_list[1].text
                # 자식 노드 dd의 두 번째에 들어가 있는 description을 text로 불러옴

                title_list.append(title)
                TitDesc_list.append( title + ' ' + description)
                # 제목과 요약내용을 붙여서 리스트에 넣음 
            except AttributeError:
                pass

        
        # 마지막 페이지 주소 확인 (다음페이지 버튼이 없으면 종료페이지로 간주)
        page = soup.find('div', {'class':'paging'})
        page_a_list = page.findAll('a')
        if '다음페이지' in str(page_a_list[-1]):
            page_num += 10
        else:
            last = True

In [20]:
Crawling('춘천', '2020.07.14')

In [75]:
TitDesc_list[:3]

['"춘천입니다"…청와대에 랜선 보고한 한성숙 네이버 대표 "네 저는 지금 춘천의 네이버 데이터센터 \'각\'에 나와있습니다. 중요한 날이라 그런지 구봉산에 시원한... 이날 조정식 민주당 정책위의장이 청와대에서 한국판 뉴딜 정책 관련 발표를 이어가던 중 춘천에 있는 한...',
 '‘학교 급식 전용’에 그친 춘천먹거리센터 춘천시가 지역에서 생산된 먹거리를 학교와 시 관계기관에 공급하겠다며 춘천 먹거리통합지원센터를 설립했습니다. 지난해 9월부터 시범 운영을 시작했는데, 애초 계획대로 잘 운영되고 있을까요. 임서영 기자가...',
 "'서민갑부' 암 수술한 아들도 먹을 수 있는 치아바타 개발로 춘천 대박 빵집 호반의 도시 춘천에서 동부 씨가 운영하는 빵집은 모르는 사람이 없을 정도로 유명하다. 특히 그의 가게는... 동부 씨는 암 수술 수 면역력이 떨어진 아들을 데리고 춘천으로 요양을 떠났다. 그동안 가족들에게 소홀했던..."]

In [21]:
TitDesc_okt = []

for item in TitDesc_list:
  item_nouns = ' '.join(okt.nouns(item))
  TitDesc_okt.append(item_nouns)

  tfidf_vectorizer = TfidfVectorizer()
  tfidf_matrix_okt = tfidf_vectorizer.fit_transform(TitDesc_okt)
TitDesc_okt[:3]
#Vectorization

['춘천 청와대 랜선 보고 성숙 네이버 대표 네 저 지금 춘천 네이버 데이터 센터 각 날 구봉산 날 조정식 민주당 정책 위 의장 청와대 한국판 뉴딜 정책 관련 발표 중 춘천',
 '학교 급식 전용 춘천 먹거리 센터 춘천시 지역 생산 먹거리 학교 시 관계 기관 공급 춘천 먹거리 통합 센터 설립 지난해 시범 운영 시작 애초 계획 운영 임서영 기자',
 '서민 갑부 암 수술 아들 수 치아바타 개발 춘천 대박 빵집 호반 도시 춘천 동부 씨 운영 빵집 사람 정도 그 가게 동부 씨 암 수술 수 면역 아들 데리 춘천 요양 그동안 가족']

In [81]:
# 토큰화 된 문장 리스트를 단어별로 split한 후 2차원 리스트로 저장
WordVoca_list = []
for item in TitDesc_okt:
    WordVoca_list.append(item.split(' '))

# split된 2차원 리스트 중에서 한 글자짜리 단어들을 모두 제외시키고 WordVoca 리스트 생성
# Word2Vec 모델 학습 데이터로 활용됨
WordVoca = []
for i in range(len(WordVoca_list)):
    element = []
    for j in range(len(WordVoca_list[i])):
        if len(WordVoca_list[i][j]) > 1:
            element.append(WordVoca_list[i][j])
    WordVoca.append(element)

In [36]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(WordVoca_list[:3]).transform(WordVoca_list[:3])
df = pd.DataFrame(te_ary, columns = te.columns_)
df.head()

,가게,가족,각,갑부,개발,계획,공급,관계,관련,구봉산,그,그동안,급식,기관,기자,날,네,네이버,뉴딜,대박,대표,데리,데이터,도시,동부,랜선,먹거리,면역,민주당,발표,보고,빵집,사람,생산,서민,설립,성숙,센터,수,수술,시,시범,시작,씨,아들,암,애초,요양,운영,위,의장,임서영,저,전용,정도,정책,조정식,중,지금,지난해,지역,청와대,춘천,춘천시,치아바타,통합,학교,한국판,호반
0,False,False,True,False,False,False,False,False,True,True,False,False,False,False,False,True,True,True,True,False,True,False,True,False,False,True,False,False,True,True,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,True,True,True,True,False,False,True,True,False,False,False,False,True,False
1,False,False,False,False,False,True,True,True,False,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,True,False,True,False,False,True,True,True,False,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,True,True,False,True,True,False,True,True,False,False
2,True,True,False,True,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,True,False,True,False,True,True,False,False,True,False,False,False,True,True,False,True,False,False,False,True,True,False,False,False,True,True,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,True,False,False,False,True


In [37]:
from mlxtend.frequent_patterns import apriori
# 지지도 support : P(A intersect B)
frequent_itemsets = apriori(df, min_support=0.05, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets[(frequent_itemsets['length'] == 2) & \
                  (frequent_itemsets['support'] >= 0.01)].sort_values(by='support',ascending=False).head(10)

KeyboardInterrupt: ignored

In [92]:
WordVoca[0]

['춘천',
 '청와대',
 '랜선',
 '보고',
 '성숙',
 '네이버',
 '대표',
 '지금',
 '춘천',
 '네이버',
 '데이터',
 '센터',
 '구봉산',
 '조정식',
 '민주당',
 '정책',
 '의장',
 '청와대',
 '한국판',
 '뉴딜',
 '정책',
 '관련',
 '발표',
 '춘천']

In [ ]:
patterns = pyfpgrowth.find_frequent_patterns(WordVoca, 2)